# Laboratorium 5 - Szyfrowanie równolegle

Przygotowanie bibiotek

In [13]:
from multiprocessing import Process, Lock, Pool, RawArray
import time
import random
import os
import ctypes
from itertools import islice

from Crypto.Cipher import DES
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

from typing import Iterable, Tuple, Any

Przydatne funkcje

In [2]:
def xor_bytearrays(a : bytearray, b : bytearray) -> bytearray:
    n = len(a)
    assert(len(b) == n)
    
    return bytearray(a[i] ^ b[i] for i in range(n))

In [3]:
a = bytearray([1, 2, 255])
b = bytearray([0, 2, 254])

xor_bytearrays(a, b)

bytearray(b'\x01\x00\x01')

In [4]:
def chunk(it : Iterable[Any], size : int) -> Iterable[Tuple[Any]]:
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

In [5]:
list(chunk(range(10), 3))

[(0, 1, 2), (3, 4, 5), (6, 7, 8), (9,)]

## Testy biblioteki `multiprocessing`

Wykożystanie blokady powstrzyma wątki przed pisanie w tym samym czasie.

In [6]:
def f(l, i):
    delay = 2 * random.random()
    time.sleep(delay)
    
    l.acquire()
    try:
        print('hello world', i)
    finally:
        l.release()

if __name__ == '__main__':
    lock = Lock()

    for num in range(10):
        Process(target=f, args=(lock, num)).start()

hello world 6
hello world 0
hello world 3
hello world 4
hello world 1
hello world 7
hello world 9
hello world 2
hello world 5
hello world 8


#### Równoległe potęgowanie

In [7]:
def f(x):
    return x**200

data = list(range(1000000))

In [38]:
%%timeit
with Pool(8) as p:
    p.map(f, data)

2.22 s ± 41.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%%timeit

# Teraz bez równoległości
list(map(f, data))

3.53 s ± 69.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Przykład procesu

In [8]:
def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())
    print()

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

main line
module name: __main__
parent process: 80754
process id: 80767

function f
module name: __main__
parent process: 80767
process id: 80782

hello bob


## Schematy działania 

### ECB - Electronic Cypher Block

Niezależne szyfrowanie

![ECB](./img/ECB.png)

### CBC - Cypher Block Chain

Kolejny blok zależy od poprzedniego

![CBC](./img/CBC.png)

## Zadania

### 1. Równolełe szyfowanie EBC

Wykorzystując przykłady napisz program implementujący równoległe szyfrowanie w trybie EBC algorytmem AES.

**Na laborce trzeba porównać prędkość równoległą z szeregową**

In [14]:
def encrypt_ECB_serial(key, plain_text):
    vector = bytearray(plain_text, 'utf-8')
    aes = AES.new(key)

    for i in range(no_blocks):
        # Select right block
        offset = i*block_size
        block = plain_text[offset:offset+block_size]

        # Why this is done 1000 times?
        for j in range(1000):
            encrypted = aes.encrypt(block)
            block = encrypted
        vector[offset:offset+block_size] = bytearray(encrypted)
    return bytes(vector)       

def decrypt_ECB_serial(key, encrypted_block):
    vector = bytearray(encrypted_block)
    aes = AES.new(key)
    for i in range(no_blocks):
        offset = i*block_size
        block = encrypted_block[offset:offset+block_size]
        for j in range(1000):
            decrypted = aes.decrypt(block)
            block = decrypted
        vector[offset:offset+block_size] = bytearray(decrypted)
    return bytes(vector) 

In [16]:
def EBC_encrypt_block(n : int) -> None:
    """Encrypts n-th block from input_data with AES and outpust to output_data"""
    
    block = bytes(input_data[n * block_size : (n+1) * block_size])
    encypted = cyper.encrypt(block)
    
    output_data[n * block_size : (n+1) * block_size] = encypted

In [18]:
def EBC_decrypt_block(n : int) -> None:
    """Decrypts n-th block from input_data with AES and outpust to output_data"""
    
    block = bytes(input_data[n * block_size : (n+1) * block_size])
    encypted = cyper.decrypt(block)
    
    output_data[n * block_size : (n+1) * block_size] = encypted

#### Test szyfrowania równoległego

In [20]:
plain_text = "alamakot"*1000
key = "haslo123" * 2

cyper = AES.new(key)
block_size = 16
no_blocks = int(len(plain_text)/block_size)

assert(len(plain_text) % block_size == 0)
assert(len(key) in {16, 24, 32})

input_data = RawArray(ctypes.c_uint8, bytearray(plain_text, 'utf-8'))
output_data = RawArray(ctypes.c_uint8, len(plain_text))

# Encryption
start_time = time.time()
with Pool(8) as pool:
    pool.map(EBC_encrypt_block, range(no_blocks))
    
print(f'Szyfrowanie zajęło   {(time.time() - start_time):.8f} s')    

encrypted = bytes(output_data)

# Decryption
input_data, output_data = output_data, input_data

start_time = time.time()
with Pool(8) as pool:
    pool.map(EBC_decrypt_block, range(no_blocks))
print(f'Deszyfrowanie zajęło {(time.time() - start_time):.8f} s')    
    

recovered = bytes(output_data)

recovered.decode()[:100]

Szyfrowanie zajęło   0.13341594 s
Deszyfrowanie zajęło 0.14493990 s


'alamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalam'

#### Porównanie prędkośći

Przygotowanie

In [25]:
plain_text = "alamakot"*1000 * 10
key = "haslo123" * 2

cyper = AES.new(key)
block_size = 16
no_blocks = int(len(plain_text)/block_size)

assert(len(plain_text) % block_size == 0)
assert(len(key) in {16, 24, 32})

input_data = RawArray(ctypes.c_ubyte, bytearray(plain_text, 'utf-8'))
output_data = RawArray(ctypes.c_ubyte, bytearray([0] * len(plain_text)))

Podejście szeregowe

In [26]:
%%timeit

encrypt_ECB_serial(key, plain_text)

2.29 s ± 77.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Podejście równoległe

In [27]:
%%timeit

with Pool(4) as pool:
    pool.map(EBC_encrypt_block, range(no_blocks))

133 ms ± 715 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
0.14340305

tmp = plain_text[:block_size]

e = cyper.encrypt(tmp)
cyper.decrypt(e)

b'alamakotalamakot'

### 2. Napisz program odszyfrowujący równolegle w trybie CBC

In [13]:
def CBC_serial_encrypt(data : bytes, key : str, iv : bytes) -> bytes:
    result = bytearray([0] * len(data))
    block_size = 16
    assert(len(data) % block_size == 0)
    assert(len(key) == 16)
    assert(len(iv) == 16)
    
    block_count = len(data) // block_size
    cyper = AES.new(key)
    
    for i in range(block_count):
        offset = i * block_size
        block = data[offset: offset + block_size]
        block = bytes(xor_bytearrays(block, iv))
        
        encrypted = cyper.encrypt(block)
        
        iv = encrypted
        result[offset: offset + block_size] = encrypted
    
    return result

In [14]:
def CBC_decrypt_block(n : int) -> None:
    """Decrypts n-th block from input_data with AES and outpust to output_data. Also uses iv variable"""
#     For the first value take iv else take cryptogram of previous block
    vector = iv if n == 0 else bytes(input_data[(n-1) * block_size : n * block_size])
    
    block = bytes(input_data[n * block_size : (n + 1) * block_size])
    
    intermidiate = cyper.decrypt(block)
    decrypted = bytes(xor_bytearrays(intermidiate, vector))
    
    output_data[n * block_size : (n + 1) * block_size] = decrypted
    

In [15]:
plain_text = "alamakot"*1000
key = "haslo123" * 2

cyper = AES.new(key)
iv = get_random_bytes(16)
block_size = 16
no_blocks = int(len(plain_text)/block_size)

assert(len(plain_text) % block_size == 0)
assert(len(key) == 16)

encrypted = CBC_serial_encrypt(plain_text.encode(), key, iv)

input_data = RawArray(ctypes.c_ubyte, encrypted)
output_data = RawArray(ctypes.c_ubyte, bytearray([0] * len(encrypted)))

start_time = time.time()
with Pool(8) as pool:
    pool.map(CBC_decrypt_block, range(no_blocks))
    
print(f'Deszyfrowanie zajęło {(time.time() - start_time):.8f} s') 

recovered = bytes(output_data)

recovered.decode()[:100]

Deszyfrowanie zajęło 0.14254093 s


'alamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalam'

### Napisz program szyfrujący w trybie CTR przy wykorzystaniu algorytmu DES

In [81]:
def CTR_encrypt_block(n : int) -> None:
    """Encrypts n-th block from input_data with DES and outpust to output_data"""
    
    block = bytes(input_data[n * block_size : (n + 1) * block_size])
    vector = nonce + n.to_bytes(4, byteorder='big')
    
    intermidiate = cyper.encrypt(vector)
    encrypted = bytes(xor_bytearrays(intermidiate, block))
    
    output_data[n * block_size : (n + 1) * block_size] = encrypted
    
#     if (n == 0):
#         print("Szyfrowanie bloku 0")
#         print("block:")
#         print(block)
#         print("vector:")
#         print(vector)
#         print("intermidiate")
#         print(intermidiate)
#         print("encrypted")
#         print(encrypted)
#         print("output")
#         print(output_data[n * block_size : (n + 1) * block_size])
#         print()

In [82]:
def CTR_decrypt_block(n : int) -> None:
    """Decrypts n-th block from input_data with DES and outpust to output_data"""
    
    block = bytes(input_data[n * block_size : (n + 1) * block_size])
    vector = nonce + n.to_bytes(4, byteorder='big')
    
    intermidiate = cyper.encrypt(vector)
    decrypted = bytes(xor_bytearrays(intermidiate, block))
    
    output_data[n * block_size : (n + 1) * block_size] = decrypted
    
#     if (n == 0):
#         print("Deszyfrowanie bloku 0")
#         print("block:")
#         print(block)
#         print("vector:")
#         print(vector)
#         print("intermidiate")
#         print(intermidiate)
#         print("encrypted")
#         print(encrypted)
#         print("output")
#         print(output_data[n * block_size : (n + 1) * block_size])
#         print()

In [83]:
plain_text = "alamakot" * 1000
key = "haslo123"

cyper = DES.new(key)
nonce = get_random_bytes(4)
block_size = 8
no_blocks = int(len(plain_text)/block_size)

assert(len(plain_text) % block_size == 0)
assert(len(key) == 8)

input_data = RawArray(ctypes.c_ubyte, bytearray(plain_text, 'utf-8'))
output_data = RawArray(ctypes.c_ubyte, bytearray([0] * len(plain_text)))

# Encryption

# Encryption
start_time = time.time()
with Pool(4) as pool:
    pool.map(CTR_encrypt_block, range(no_blocks))
# print(f'Szyfrowanie zajęło   {(time.time() - start_time):.8f} s')  
  
encrypted = bytes(output_data)
# print(encrypted[:100])

input_data, output_data = output_data, input_data

# Decryption
start_time = time.time()
with Pool(4) as pool:
    pool.map(CTR_decrypt_block, range(no_blocks))
# print(f'Deszyfrowanie zajęło   {(time.time() - start_time):.8f} s')  
    
recovered = bytes(output_data)

print("Fragmen tekstu jawnego")
print(plain_text[:100])

print("Fragmen kryptogramu")
print(encrypted[:100])

print("Fragmen odszyfrowanego tekstu")
print(recovered[:100])

# encrypted.decode()[:100]

Fragmen tekstu jawnego
alamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalam
Fragmen kryptogramu
b'R\xbb\x86\x85\xb6\xb5\xda\xf06]\xeay\x83\xfd\x8d\xc0\xb3&\x01\x9cy7\xc9^\x17c\x8b3\xdc=q\xccor1A\x9b{0\xb3\x10\x90:(\x17R:{\xfao\xf8W\x125\xc3:F!R\xe0\xc4\xa7\xc1\xeerb\xe0\x88kO,r\xbc^\x85pk\x95\xa2\xe6C\x8a\xb8\xe5Tu\xf6\xaf\x81\xef\xe5G\x0bo\xe0\xbf\xc9m)\x84'
Fragmen odszyfrowanego tekstu
b'alamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalamakotalam'


In [69]:
!pip3 install pycryptodomex

## Zadnie 3 Laborki

Znajdź poprawną kombinację kluca i "sdi" (nonce) ze słownika w pliku dict.txt, który pozwoli na poprawne odszyfrowanie kryptogramu.  
(dict['updatet']. Kryptogram zaszyfrowano w trypbie AES MODE_CCM analogicznie do _exampleCCM.py_

TIP: wykorzystaj dict['tag']

Przykład

In [68]:
#pip3 install pycryptodomex

from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes


def encrypt_CCM(data, key, nonce):
	cipher = AES.new(key, AES.MODE_CCM, nonce)
	ciphertext = cipher.encrypt(data)
	tag = cipher.digest()
	return ciphertext, tag

def decrypt_CCM(ciphertext,key,nonce,tag):
	cipher = AES.new(key, AES.MODE_CCM, nonce)

	plaintext = cipher.decrypt(ciphertext)

	try:
		cipher.verify(tag)
	except ValueError:
		return False

	return plaintext


message = b'\xe1~\x8a\xd1\x96\x84\xc38v\xc7\x8e\x04\x1f\x12\x96p#\xaa\x11t\x17\x9b\x93(/\x7fg\xfa\x9b.\xdd\xe3\x02\xef<\xca\xe7\xac'

key = get_random_bytes(16)
nonce = get_random_bytes(11)
ciphertext,tag = encrypt_CCM(message,key,nonce)


corrupted_nonce = get_random_bytes(11)

#odszyfrowanie z błędną "solą""
decrypted = decrypt_CCM(ciphertext,key,corrupted_nonce,tag)

#odszyfrowanie poprawne
#decrypted = decrypt_CCM(ciphertext,key,nonce,tag)

if decrypted: 
	print("Odszyfrowano! Twoja wiadomość:")
	print(decrypted)
else:
	print("Niepoprawne odszyfrowanie!")



ModuleNotFoundError: No module named 'Cryptodome'